# Plain Stress
Calculate stress's load and stiffness matrix

## Custom data
![](https://raw.githubusercontent.com/linnil1/2019FEM/master/images/PlainStress.png)

* E: Elastic
* v: Poisson ratio
* nodes: three (x, y) locations
* thick: thickness
* force: external concentrated force
* P: external distributed force(pressure)

In [1]:
import numpy as np

E = 200e5
# posisson ratio
v = 0.32
nodes = [[2.25, 0.75],
         [2.40, 1.65],
         [1.5,  1.0 ]]
thick = 0.3
# Force at i, j, k
forces = [0, 0, -500 * np.cos(np.deg2rad(50)), -500 * np.sin(np.deg2rad(50)), 0, 0]
# Pressure at ij, jk, ki
P = [0, 0, 0, 0, 1200, 0]

In [2]:
nodes = np.array(nodes)
forces = np.array(forces)
P = np.array(P)

def getK():
    V = E / (1 - v ** 2) * np.array([
            [1, v, 0],
            [v, 1, 0],
            [0, 0, (1 - v) / 2]])
    dpos = np.roll(nodes, -1, axis=0) - np.roll(nodes, -2, axis=0)
    dpos[:, 0] *= -1
    A = np.abs(np.cross(dpos[0], dpos[1])) / 2
    B =  np.zeros([3, 6])
    B[0, 0::2] = dpos[:, 1]
    B[1, 1::2] = dpos[:, 0]
    B[2, 0::2] = dpos[:, 0]
    B[2, 1::2] = dpos[:, 1]
    B = B / 2 / A

    K = A * thick * (B.T).dot(V).dot(B)
    return K

def getF():
    F = np.zeros(6)
    for i in range(3):
        j = (i + 1) % 3
        l = np.sqrt(((nodes[i] - nodes[j]) ** 2).sum())
        i *= 2
        j *= 2
        F[[i, i + 1]] += forces[[i, i + 1]]
        F[[i,     j    ]] += thick / 2 * l * P[i    ]
        F[[i + 1, j + 1]] += thick / 2 * l * P[i + 1]
    return F

print("K")
print(getK())
print("F")
print(getF())

K
[[ 3273759.26447134 -1811145.51083591  -314288.39478375   372924.28933296
  -2959470.86968759  1438221.22150295]
 [-1811145.51083591  4473449.66694812   439769.20911905 -2907167.6517497
   1371376.30171686 -1566282.01519842]
 [ -314288.39478375   439769.20911905  1190308.65934891   580495.35603715
   -876020.26456516 -1020264.56515621]
 [  372924.28933296 -2907167.6517497    580495.35603715  2738296.27544798
   -953419.64537011   168871.37630172]
 [-2959470.86968759  1371376.30171686  -876020.26456516  -953419.64537011
   3835491.13425274  -417956.65634675]
 [ 1438221.22150295 -1566282.01519842 -1020264.56515621   168871.37630172
   -417956.65634675  1397410.63889671]]
F
[ 142.30249471    0.         -321.39380484 -383.02222156  142.30249471
    0.        ]
